# Grab the list of files for pre-fire date

# Grab list of files for post-fire date

# Find files with specific bands

# Open images


# Get list of shapefiles
	
# For each shapefile:
		
## Clip the images
## Running dNBR
## Calculate dNBR for whole basin
## Save dNBR image
## Running prediction    
## Calculate prediction for whole basin
## Save prediction image
## Create 3 - panel image

# END

In [3]:
# IMPORT NEEDED LIBRARIES
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
from numpy import asarray
import rioxarray as rxr
import rasterio as rio
from rasterio.enums import Resampling
import geopandas as gpd
import xarray as xr
import pandas as pd
import glob
import os
import fnmatch
import math
import statistics as st
from math import e
#from multispectral_functions import *

In [141]:
# SET IMAGE AND SHAPEFILE DIRECTORIES

# Pre-fire & RGB directory
dirImgLoc = '/mnt/nfs/lss/meerdink/home/skzebarth/masters/sentinel_2b/raw/S2B_MSIL1C_20171128T184719_N0206_R070_T11SKU_20171128T203222.SAFE/GRANULE/L1C_T11SKU_A003814_20171128T184714/IMG_DATA/'

# Post-fire directory
dirImgLocPost = '/mnt/nfs/lss/meerdink/home/skzebarth/masters/sentinel_2b/raw/S2B_MSIL1C_20171228T184749_N0206_R070_T11SKU_20171228T221718.SAFE/GRANULE/L1C_T11SKU_A004243_20171228T184751/IMG_DATA/'

# Shapefile directory, ensure correct number of shapefiles for analysis
dirPolyLoc = '/mnt/nfs/lss/meerdink/home/skzebarth/masters/shapefiles/'




# Output folders
rgb_out = '/mnt/nfs/lss/meerdink/home/skzebarth/masters/batch_output/rgb/'
nbr_out = '/mnt/nfs/lss/meerdink/home/skzebarth/masters/batch_output/nbr/'
dnbr_out = '/mnt/nfs/lss/meerdink/home/skzebarth/masters/batch_output/dnbr/'
usgs_dnbr_out = '/mnt/nfs/lss/meerdink/home/skzebarth/masters/batch_output/usgs_dnbr/'
usgs_pha_out = '/mnt/nfs/lss/meerdink/home/skzebarth/masters/batch_output/usgs_pha'
ppp_out = '/mnt/nfs/lss/meerdink/home/skzebarth/masters/batch_output/ppp/'
fig_out = '/mnt/nfs/lss/meerdink/home/skzebarth/masters/batch_output/fig/'
general = '/mnt/nfs/lss/meerdink/home/skzebarth/masters/batch_output/general/'

In [137]:
# ALL FUNCTIONS

def nbr_clip(dir, 
             shape, 
             plot=False, 
             save=False):
    '''Takes in image and clips the raster to study basins.
        
        CLIP OPTIONS:
        all_touched (boolean, OPTIONAL): If True, all pixels touched by geometries
        will be burned in. If false, only pixels whose center is within the polygon 
        or that are selected by Bresenham’s line algorithm will be burned in.
        
        drop (bool, optional): If True, drop the data outside of the extent of the 
        mask geoemtries Otherwise, it will return the same raster with the data masked. 
        Default is True.
        
        invert (boolean, optional): If False, pixels that do not overlap shapes will be
        set as nodata. Otherwise, pixels that overlap the shapes will be set as nodata. 
        False by default.'''
    
    NIR_path = glob.glob(dir + '/*B8A.jp2')
    NIR = rxr.open_rasterio(NIR_path[0], masked=True)
    SWIR_path = glob.glob(dir + '/*B12.jp2')
    SWIR = rxr.open_rasterio(SWIR_path[0], masked=True)
    
    name = os.path.basename(NIR_path[0])
    name_split = name.split('_')
    val = name_split[1]
    date = val[:8]
    sdc = f"{val[4:6]}/{val[6:8]}/{val[0:4]}"
    
    shape = gpd.read_file(os.path.join(dirPolyLoc, filename))
    b_name = filename.split('.')
    basin = b_name[0]
    basin_NIR = NIR.rio.clip(shape.geometry.values,
                                shape.crs,
                                all_touched = False,
                                drop = True,
                                invert = False)
    basin_SWIR = SWIR.rio.clip(shape.geometry.values,
                                shape.crs,
                                all_touched = False,
                                drop = True,
                                invert = False)
    
    #basin_clip.plot()
    
    
    nbr = (basin_NIR - basin_SWIR) / (basin_NIR + basin_SWIR)
    if plot == True:
        nbr.plot(
            cmap = 'gray',
            vmax = 1,
            vmin = -1)
        plt.title(f"Basin {basin} Normalized Burn Ratio: {sdc}")
   
    if save == True:
        outname = f"{date}_{basin}_nbr.tif"
        nbr.rio.to_raster(os.path.join(nbr_out, outname), driver='GTIFF')
    
    return nbr
#####
def dnbr(pre_fire, 
         post_fire, 
         plot=False, 
         save=False):
    '''Takes in nbr functions outputs and calculates the differenced normalized burn ratio.
        This function outputs individual basin results.'''
    
    dnbr = pre_fire - post_fire
    
    if plot == True:
        dnbr.plot(
            cmap = 'gray',
            vmax = 1,
            vmin = -1)
        plt.title(f"Basin {basin} Differenced Normalized Burn Ratio")
        
    if save == True:
        outname = f"{basin}_dnbr.tif"
        dnbr.rio.to_raster(os.path.join(dnbr_out, outname), driver='GTIFF')
        
    return dnbr

def avg_dnbr(pre_fire, 
         post_fire, 
         plot=False, 
         save=False):
    '''Takes in nbr functions outputs and calculates the differenced normalized burn ratio.
        This function outputs individual basin results.'''
    
    dnbr = pre_fire - post_fire
    dnbr_flat = dnbr.values.flatten()
    pixarr = np.empty((dnbr_flat.shape[0]), dtype= float)
    pixarr[:] = np.nan
    x = np.where(np.isnan(dnbr_flat) == False)
    dnbrdata = dnbr_flat[x]
    p1 = []
    for i in dnbrdata:
        individual_basin = x_values_df.index[x_values_df['basin_id'] == int(basin)]
        x_2 = x_values_df['x2'][individual_basin]
        p1.append(x_2)
    
    # Append average array, flatten data, reshape into image
    avg_arr = np.array(p1)
    a_final = avg_arr.flatten()
    pixarr[x] = a_final
    pixarrImg = pixarr.reshape((dnbr.shape[1], dnbr.shape[2]))
    out_dnbr = xr.Dataset()
    out_dnbr = xr.DataArray(pixarrImg, dims = ('y', 'x'),
                            coords = {'x': dnbr.coords['x'],
                                      'y': dnbr.coords['y']})

    if plot == True:
        out_dnbr.plot(
            cmap = 'gray',
            vmax = 1,
            vmin = -1)
        plt.title(f"Basin {basin} USGS Differenced Normalized Burn Ratio")
        
    if save == True:
        outname = f"{basin}_usgs_dnbr.tif"
        out_dnbr.rio.to_raster(os.path.join(usgs_dnbr_out, outname), driver='GTIFF')
        
    return out_dnbr
        
def pha(dnbr, 
          plot=False, 
          save=False):
    
    # Coefficients for Southern California
    b = -3.63
    b_1 = 0.41
    b_2 = 0.67
    b_3 = 0.7
    
    
    dnbr_flat = dnbr.values.flatten()
    pixarr = np.empty((dnbr_flat.shape[0]), dtype= float)
    pixarr[:] = np.nan
    x = np.where(np.isnan(dnbr_flat) == False)
        
    # Getting the data from dnbr_flat
    dnbrdata = dnbr_flat[x]
        
    # Matching index of x_values_df[basin_id] to shapefile
    individual_basin = x_values_df.index[x_values_df['basin_id'] == int(basin)]
        
    # Pulling individual values from x_1 and x_3, creating pixel values for x_2 denoted by s & s1
    x_1 = x_values_df['x1'][individual_basin]
    x_2 = x_values_df['x2'][individual_basin]
    x_3 = x_values_df['x3'][individual_basin]
       
    # Per pixel analysis, probability stored in empty list p1
    p1 = []
    for i in dnbrdata:
        lnx =  b + (b_1 * x_1 * val) + (b_2 * x_2 * val) + (b_3 * x_3 * val)
        prob = (e ** lnx) / (1.0 + e ** lnx)
        p1.append(prob)
        
    # Append probability array, flatten data, reshape into image
    prob_arr = np.array(p1)
    p_final = prob_arr.flatten()
    pixarr[x] = p_final
    pixarrImg = pixarr.reshape((dnbr.shape[1], dnbr.shape[2]))
    out_dnbr = xr.Dataset()
    out_dnbr = xr.DataArray(pixarrImg, dims = ('y', 'x'),
                            coords = {'x': dnbr.coords['x'],
                                      'y': dnbr.coords['y']})

    if plot == True:
        out_dnbr.plot(
            cmap = 'gnuplot',
            vmax = 1,
            vmin = 0)
        plt.title(f"Basin {basin} Initiation Probablilty, R = {int(val*4)}mm/hr")
    
    if save == True:
        outname = f"{basin}_usgsP_{int(val*4)}mmhr.tif"
        out_dnbr.rio.to_raster(os.path.join(usgs_pha_out, outname), driver='GTIFF')
        print(f"Basin {basin} {int(val*4)}mm/hr prob analysis saved as geotiff")
        
        print(f"Basin {basin} {int(val*4)}mm/hr prob analysis complete")
    return out_dnbr

def ppp(dnbr, 
          plot=False, 
          save=False):
    
    # Coefficients for Southern California
    b = -3.63
    b_1 = 0.41
    b2 = 0.67
    b_2 = np.array(b2)
    b_3 = 0.7
    
    
    dnbr_flat = dnbr.values.flatten()
    pixarr = np.empty((dnbr_flat.shape[0]), dtype= float)
    pixarr[:] = np.nan
    x = np.where(np.isnan(dnbr_flat) == False)
        
    # Getting the data from dnbr_flat
    dnbrdata = dnbr_flat[x]
        
    # Matching index of x_values_df[basin_id] to shapefile
    individual_basin = x_values_df.index[x_values_df['basin_id'] == int(basin)]
        
    # Pulling individual values from x_1 and x_3, creating pixel values for x_2 denoted by s & s1
    x_1 = x_values_df['x1'][individual_basin]
    x_3 = x_values_df['x3'][individual_basin]
    s = np.multiply(dnbrdata, b_2)
    s1 = np.multiply(s, val)
    # s1 multiplication assumes when function is called that val is defined globally in code.
    
    # Per pixel analysis, probability stored in empty list p1
    p1 = []
    for i in s1:
        lnx =  b + (b_1 * x_1 * val) + i + (b_3 * x_3 * val)
        prob = (e ** lnx) / (1.0 + e ** lnx)
        p1.append(prob)
        
    # Append probability array, flatten data, reshape into image
    prob_arr = np.array(p1)
    p_final = prob_arr.flatten()
    pixarr[x] = p_final
    pixarrImg = pixarr.reshape((dnbr.shape[1], dnbr.shape[2]))
    out_dnbr = xr.Dataset()
    out_dnbr = xr.DataArray(pixarrImg, dims = ('y', 'x'),
                            coords = {'x': dnbr.coords['x'],
                                      'y': dnbr.coords['y']})

    if plot == True:
        out_dnbr.plot(
            cmap = 'gnuplot',
            vmax = 1,
            vmin = 0)
        plt.title(f"Basin {basin} Per Pixel Initiation Probablilty, R = {int(val*4)}mm/hr")
    
    if save == True:
        outname = f"{basin}_ppp_{int(val*4)}mmhr.tif"
        out_dnbr.rio.to_raster(os.path.join(ppp_out, outname), driver='GTIFF')
        print(f"Basin {basin} {int(val*4)}mm/hr per pixel analysis saved as geotiff")
        
        print(f"Basin {basin} {int(val*4)}mm/hr per pixel analysis complete")
    return out_dnbr

#####

def rgb(dir, 
        shape, 
        plot=False, 
        save=False):
    
    def normalize(array):
        """Normalizes numpy arrays into scale 0.0 - 1.0"""
        array_min, array_max = array.min(), array.max()
        return ((array - array_min)/(array_max - array_min))
    def resample(rst):
        scale_factor = 1/2
        new_width = rst.rio.width * scale_factor
        new_height = rst.rio.height * scale_factor
        sampled = rst.rio.reproject(rst.rio.crs, 
                                    shape=((int(new_height), int(new_width))),
                                    resampling = Resampling.nearest)
        return sampled
    
    # Set RGB Bands,Sentinel-2 is red(B04), green(B03), blue(B02)
    imageRed = glob.glob(dir + '/*B04.jp2')
    imageGreen = glob.glob(dir + '/*B03.jp2')
    imageBlue = glob.glob(dir + '/*B02.jp2')
    name = os.path.basename(imageRed[0])
    name_split = name.split('_')
    val = name_split[1]
    date = val[:8]
    sdc = f"{val[4:6]}/{val[6:8]}/{val[0:4]}"
    imageRed_open = rxr.open_rasterio(imageRed[0], masked=True)
    imageBlue_open = rxr.open_rasterio(imageBlue[0], masked=True)
    imageGreen_open = rxr.open_rasterio(imageGreen[0], masked=True)
    
    shape = gpd.read_file(os.path.join(dirPolyLoc, filename))
    b_name = filename.split('.')
    basin = b_name[0]
    
    redClip = imageRed_open.rio.clip(shape.geometry.values, 
                                     shape.crs, 
                                     all_touched=False, 
                                     drop=True, 
                                     invert=False)
    blueClip = imageBlue_open.rio.clip(shape.geometry.values, 
                                       shape.crs, 
                                       all_touched=False, 
                                       drop=True, 
                                       invert=False)
    greenClip = imageGreen_open.rio.clip(shape.geometry.values, 
                                         shape.crs, 
                                         all_touched=False, 
                                         drop=True, 
                                         invert=False)
    
    # Normalize clip so rgb display is bright enough
    redn = normalize(redClip)
    bluen = normalize(blueClip)
    greenn = normalize(greenClip)
    
    # Resample from 10m to 20m spatial resolution
    red_rs = resample(redn)
    blue_rs = resample(bluen)
    green_rs = resample(greenn)
    
    # Pull band information from each array
    redData = red_rs.data[0,:,:]
    blueData = blue_rs.data[0,:,:]
    greenData = green_rs.data[0,:,:]
    
    # Create and plot mask
    dataPlot = np.zeros((redData.shape[0],redData.shape[1],3))
    dataPlot[:,:,0] = redData
    dataPlot[:,:,1] = greenData
    dataPlot[:,:,2] = blueData
    dataPlotMask = np.ma.masked_where(np.isnan(dataPlot), dataPlot)
    img = xr.Dataset()
    img = xr.DataArray(dataPlotMask,
                           dims = ('y', 'x', 'band'),
                           coords ={'y': red_rs.coords['y'], 
                                    'x': red_rs.coords['x'],
                                    'band': ["red", "green", "blue"]})
    
    
    # Transpose array for geotiff output
    out_img = img.transpose('band', 'y', 'x')
    
    if plot == True:
        img.plot.imshow()
        plt.title(f'Basin {basin} RGB: {sdc}')
        plt.show()
    #Saving
    if save == True:
        outname = f"{basin}_{date}_rgb.tif"
        out_img.rio.to_raster(os.path.join(rgb_out, outname), driver='GTIFF')
    print(f"Basin {basin} RGB combine complete")
    return img

####
def dnbr_hist(data):
    avg = np.mean(data)
    fig, ax = plt.subplots()
        
    data.plot.hist(ax=ax,
                   color='c',
                   edgecolor='k',
                   bins=20)
    plt.axvline(avg, color='k', linestyle='dashed', linewidth=2)
    min_ylim, max_ylim = plt.ylim()
    plt.text(avg*1.05, max_ylim*0.95, 'Mean: {:.3f}'.format(avg))   
    ax.set(
        title = f"Basin {basin} dNBR Distribution",
        xlabel='dNBR',
        ylabel='Frequency')
    plt.show()
    return fig

def ppp_hist(data):
    avg = np.mean(data)
    fig, ax = plt.subplots()
        
    data.plot.hist(ax=ax,
                   color='c',
                   edgecolor='k',
                   bins=20)
    plt.axvline(avg, color='k', linestyle='dashed', linewidth=2)
    min_ylim, max_ylim = plt.ylim()
    plt.text(avg*1.05, max_ylim*0.95, 'Mean: {:.3f}'.format(avg))   
    ax.set(
        title = f"Basin {basin} Per Pixel Probability Distribution",
        xlabel='Debris Flow Initiation Likelihood',
        ylabel='Frequency')
    #plt.show()
    return fig

#####



def fig_1x3(tci, 
             bs, 
             prob,
             savefig=False):
    # Create figures
    fig, ax = plt.subplots(1, 3, 
                           figsize=(14,4), 
                           gridspec_kw={'width_ratios':[1, 1.2, 1.2]})
    fig.suptitle(f'Basin {basin} with {int(val*4)}mm/hr Design Storm', fontsize=15, fontweight='bold')
    
    # RGB imagery
    tci.plot.imshow(ax=ax[0])
    ax[0].axis('off')
    ax[0].set_title('RGB', y=-.1)
    
    # dNBR imagery
    value1 = np.mean(bs)
    mean1 = value1.item()
    bs.plot(ax=ax[1], 
            cmap = 'gray',
            vmax = 1,
            vmin = -1)
    ax[1].set_title(f'Basin dNBR: {mean1:.3g}', y=-.1)
    ax[1].axis('off')
    
    # PPP imagery
    value2 = np.mean(prob)
    mean2 = value2.item()
    prob.plot(ax=ax[2], 
            cmap = 'gnuplot',
            vmax = 1,
            vmin = 0)
    ax[2].set_title(f'Basin P: {mean2:.3g}', y=-.1)
    ax[2].axis('off')
    if savefig == True:
       plt.savefig(os.path.join(fig_out, f'{basin}_{int(val*4)}mmhr_vis_1by3.png'))
    plt.show()
    return fig

#####

def fig_1x2(dnbr, 
            ppp, 
            savefig=False):
     fig, ax = plt.subplots(1, 2, 
                           figsize=(14,5))
     fig.suptitle(f'Basin {basin} Data Distribution with {int(val*4)}mm/hr Design Storm', fontsize=15, fontweight='bold')
    
     # dNBR histogram
    
     value1 = np.mean(dnbr)
     dnbravg = value1.item()
     dnbr.plot.hist(
                    color= 'c',
                    edgecolor='k',
                    bins=20,
                    ax=ax[0])
     ax[0].set(
         title = 'dNBR',
         xlabel=f'Mean: {dnbravg: .3g}',
         ylabel='Frequency')
     ax[0].axvline(dnbravg, 
                 color='r', 
                 linestyle='solid', 
                 linewidth=2,
                 ) 
     
     
     
     value2 = np.mean(ppp)
     pppavg = value2.item()
     ppp.plot.hist(
                    color='c',
                    edgecolor='k',
                    bins=20,
                    ax=ax[1])
     
     ax[1].axvline(pppavg, 
                 color='r', 
                 linestyle='solid', 
                 linewidth=2,
                 )  
     ax[1].set(
        title= 'Per Pixel Probability',
        xlabel=f'Mean: {pppavg: .3g}')
        
     if savefig == True:
        plt.savefig(os.path.join(fig_out, f'{basin}_{int(val*4)}mmhr_hist.png'))
     plt.show()
     return fig

def fig_1x4(tci,
             tci2,
             bs, 
             prob,
             savefig=False):
    # Create figures
    fig, ax = plt.subplots(1, 4, 
                           figsize=(18,4), 
                           gridspec_kw={'width_ratios':[1, 1, 1.2, 1.2]})
    fig.suptitle(f'Basin {basin} with {int(val*4)}mm/hr Design Storm', fontsize=15, fontweight='bold')
    
    # RGB imagery
    tci.plot.imshow(ax=ax[0])
    ax[0].axis('off')
    ax[0].set_title('Pre-fire', y=-.1)
    
    tci2.plot.imshow(ax=ax[1])
    ax[1].axis('off')
    ax[1].set_title('Post-fire', y=-.1)
    
    # dNBR imagery
    value1 = np.mean(bs)
    mean1 = value1.item()
    bs.plot(ax=ax[2], 
            cmap = 'gray',
            vmax = 1,
            vmin = -1)
    ax[2].set_title(f'Basin dNBR: {mean1:.3g}', y=-.1)
    ax[2].axis('off')
    
    # PPP imagery
    value2 = np.mean(prob)
    mean2 = value2.item()
    prob.plot(ax=ax[3], 
            cmap = 'gnuplot',
            vmax = 1,
            vmin = 0)
    ax[3].set_title(f'Basin P: {mean2:.3g}', y=-.1)
    ax[3].axis('off')
    if savefig == True:
       plt.savefig(os.path.join(fig_out, f'{basin}_{int(val*4)}mmhr_vis_by4.png'))
    plt.show()
    return fig

def fig_6(tci, tci2, dnbr, ppp, p, savefig=False):
     individual_basin = x_values_df.index[x_values_df['basin_id'] == int(basin)]  
     usgs_x2 = float(x_values_df['x2'].iloc[individual_basin])
     value1 = np.mean(dnbr)
     mean1 = value1.item()
     value2 = np.mean(ppp)
     mean2 = value2.item()
     vals, counts = np.unique(dnbr, return_counts=True)
     mode_value = np.argwhere(counts == np.max(counts))
     fig = plt.figure(figsize = (15,8))
     gs = gridspec.GridSpec(2, 3, 
                            width_ratios=[1,1.2,1.2])
     fig.suptitle(f'Basin {basin}, {int(val*4)}mm/hr Design Storm', fontsize=15, fontweight='bold')
    
     # Create axes for top three images
     ax1 = fig.add_subplot(gs[0, 0])
     ax2 = fig.add_subplot(gs[0, 1])
     ax3 = fig.add_subplot(gs[0, 2])

     # Create axes for bottom two images
     ax4 = fig.add_subplot(gs[1, 0])
     ax5 = fig.add_subplot(gs[1, 1])
     ax6 = fig.add_subplot(gs[1, 2])
     
     fig.subplots_adjust(wspace=.25)
     # Plot images on axes
     tci.plot.imshow(ax=ax1)
     ax1.axis('off')
     ax1.set_title('Pre-fire', y=-.1)
     
     dnbr.plot(ax=ax2, 
            cmap = 'gray',
            vmax = 1,
            vmin = -1)
     ax2.set_title(f'Basin dNBR: {mean1:.3g}', y=-.1)
     ax2.axis('off')
     
     ppp.plot(ax=ax3, 
            cmap = 'gnuplot',
            vmax = 1,
            vmin = 0)
     ax3.set_title(f'Basin P: {mean2:.3g}', y=-.1)
     ax3.axis('off')
     
     tci2.plot.imshow(ax=ax4)
     ax4.axis('off')
     ax4.set_title('Post-fire', y=-.17)   
    
     dnbr.plot.hist(
                    color= 'c',
                    edgecolor='k',
                    bins=20,
                    ax=ax5)
     ax5.set(
         title='dNBR Distribution',
         xlabel=f'USGS x2: {usgs_x2: .3g}, Study x2: {mean1: .3g}',
         ylabel='Frequency')
     ax5.axvline(mean1, 
                 color='r', 
                 linestyle='solid', 
                 linewidth=2,
                 )
     ax5.axvline(usgs_x2, 
                 color='y', 
                 linestyle='solid', 
                 linewidth=2,
                 )
     ax5.axvline(mode_value, 
                 color='k', 
                 linestyle='solid', 
                 linewidth=2,
                 )
     ax5.set_xlim([-1,1])
     ppp.plot.hist(
                    color='c',
                    edgecolor='k',
                    bins=20,
                    ax=ax6)
     
     ax6.axvline(mean2, 
                 color='r', 
                 linestyle='solid', 
                 linewidth=2,
                )
     ax6.axvline(p, 
                 color='y', 
                 linestyle='solid', 
                 linewidth=2,
                 )  
     ax6.set(
        title='Probability Distribution',
        xlabel=f'USGS P:{p: .3g}, PP avg: {mean2: .3g}')
     ax6.set_xlim([0,1])
     
     
     if savefig == True:
         plt.savefig(os.path.join(fig_out, f'{basin}_{int(val*4)}mmhr_by6.png'))   
     plt.show()
     #plt.close()
     return        
     
        
def p_choice(val):
    individual_basin = x_values_df.index[x_values_df['basin_id'] == int(basin)]
    if val == 3.0:
        p = float(x_values_df['P_3'].iloc[individual_basin])
    elif val == 4.0:
        p = float(x_values_df['P_4'].iloc[individual_basin])
    elif val == 5.0:
        p = float(x_values_df['P_5'].iloc[individual_basin])
    elif val == 6.0:
        p = float(x_values_df['P_6'].iloc[individual_basin])
    elif val == 7.0:
        p = float(x_values_df['P_7'].iloc[individual_basin])
    elif val == 8.0:
        p = float(x_values_df['P_8'].iloc[individual_basin])
    elif val == 9.0:
        p = float(x_values_df['P_9'].iloc[individual_basin])
    elif val == 10.0:
        p = float(x_values_df['P_10'].iloc[individual_basin])
    return p

  
def fig_8(tci, tci2, avg_dnbr, dnbr, pha, ppp, p, savefig=False):
     individual_basin = x_values_df.index[x_values_df['basin_id'] == int(basin)]  
     usgs_x2 = float(x_values_df['x2'].iloc[individual_basin])
     value1 = np.mean(dnbr)
     mean1 = value1.item()
     value2 = np.mean(ppp)
     mean2 = value2.item()
     
     fig, ax = plt.subplots(2, 4, 
                           figsize=(18,10), 
                           gridspec_kw={'width_ratios':[1, 1.2, 1.2, 1.8]})
     
     fig.suptitle(f'Basin {basin}, {int(val*4)}mm/hr Design Storm', fontsize=15, fontweight='bold')
     ax1 = ax[0,0] 
     ax2 = ax[0,1]
     ax3 = ax[0,2]
     ax4 = ax[0,3]
     ax5 = ax[1,0]
     ax6 = ax[1,1]
     ax7 = ax[1,2]
     ax8 = ax[1,3]
     fig.tight_layout
     # Plot images on axes
     tci.plot.imshow(ax=ax1)
     ax1.axis('off')
     ax1.set_title('Pre-fire')
        
     avg_dnbr.plot(ax=ax2, 
               cmap = 'gray',
               vmax = 1,
               vmin = -1)
     ax2.set_title(f'A) USGS dNBR mean: {usgs_x2: .3g}')
     ax2.axis('off')
     
    
     dnbr.plot(ax=ax3, 
            cmap = 'gray',
            vmax = 1,
            vmin = -1)
     ax3.set_title(f'B) Study dNBR mean: {mean1:.3g}')
     ax3.axis('off')
     dnbr.plot.hist(
                    color= 'c',
                    edgecolor='k',
                    bins=20,
                    ax=ax4)
     ax4.set(
         title='dNBR Distribution',
         #xlabel=f'USGS x2: , Study x2: {mean1: .3g}',
         ylabel='Frequency')
     ax4.set_xlim([-1,1])
     ax4.axvline(usgs_x2, 
                 color='b', 
                 linestyle='solid', 
                 linewidth=2,
                 label = 'A)'
                 )
     ax4.axvline(mean1, 
                 color='r', 
                 linestyle='solid', 
                 linewidth=2,
                 label = 'B)'
                 )
     
     ax4.yaxis.set_label_position('right')
     ax4.yaxis.tick_right()
     ax4.legend(loc='best')
        
     tci2.plot.imshow(ax=ax5)
     ax5.axis('off')
     ax5.set_title('Post-fire')  
     
     pha.plot(ax=ax6, 
            cmap = 'gnuplot',
            vmax = 1,
             vmin = 0)
     ax6.set_title(f'C) USGS P:{p: .3g}')
     ax6.axis('off')
    
     ppp.plot(ax=ax7, 
            cmap = 'gnuplot',
            vmax = 1,
            vmin = 0)
     ax7.set_title(f'D) Pixel P mean: {mean2:.3g}')
     ax7.axis('off')
     
     ppp.plot.hist(
                    color='c',
                    edgecolor='k',
                    bins=20,
                    ax=ax8)
     ax8.axvline(p, 
                 color='b', 
                 linestyle='solid', 
                 linewidth=2,
                 label = 'C)'
                 ) 
     ax8.axvline(mean2, 
                 color='r', 
                 linestyle='solid', 
                 linewidth=2,
                 label = 'D)'
                )
      
     ax8.set(
        title='Probability Distribution',
        #xlabel=f'PP avg: {mean2: .3g}',
        ylabel='Frequency')
     ax8.yaxis.set_label_position('right')
     ax8.yaxis.tick_right()
     ax8.set_xlim([0,1])
     ax8.legend(loc='best')
     
     if savefig == True:
         plt.savefig(os.path.join(fig_out, f'{basin}_{int(val*4)}mmhr_2x4.png'))   
     #plt.show()
     plt.close()
     return      

In [143]:
choice = input('''Input your choice.
                  1: Run all basins with R value = 24 mm/hr.
                  2: Run all basins with all R values.
                  ''')
x_values_df = pd.read_csv('/mnt/nfs/lss/meerdink/home/skzebarth/masters/basin_data/ma_data.csv', 
                          delimiter=',')

for filename in sorted(os.listdir(dirPolyLoc)):
    if filename.endswith('.shp'):
        b_name = filename.split('.')
        basin = b_name[0]
        
        shape = gpd.read_file(os.path.join(dirPolyLoc, filename))
        rgb_img = rgb(dirImgLoc, shape, plot=False, save=False)
        rgb_img2 = rgb(dirImgLocPost, shape, plot=False, save=False)
        pre_fire_img = nbr_clip(dirImgLoc, shape, plot=False, save=False)
        print(f'Prefire imagery and basins NBR analysis complete for basin {basin}.')
        post_fire_img = nbr_clip(dirImgLocPost, shape, plot=False, save=False)
        print(f'Postfire imagery and basins NBR analysis complete for basin {basin}.')
        avg_dnbr_img = avg_dnbr(pre_fire_img, post_fire_img, plot=False, save=True)
        dnbr_img = dnbr(pre_fire_img, post_fire_img, plot=False, save=False)
        print(f'DNBR analysis complete for basin {basin}.')
        if choice == '1':
            R = [6.0]
            for val in R:
                p = p_choice(val)
                pha_img = pha(dnbr_img, plot=False, save=False)
                ppp_img = ppp(dnbr_img, plot=False, save=False)
                print(f'Per Pixel Analysis complete for basin {basin}.')
                #fig = fig_1x3(rgb_img, dnbr_img, ppp_img, savefig=False)
                #hist = fig_1x2(dnbr_img, ppp_img, savefig=False)
                #fig2 = fig_1x4(rgb_img, rgb_img2, dnbr_img, ppp_img, savefig=False)
                #fig1 = fig_6(rgb_img, rgb_img2, dnbr_img, ppp_img, p, savefig=False)
                fig = fig_8(rgb_img, rgb_img2, avg_dnbr_img, dnbr_img,
                            pha_img, ppp_img, p, savefig=False)
                
            #print(f'Basin {basin} Debris Flow Likelihood analysis completed R value equicalent to 24mm/hr')
        if choice == '2':
            R = [3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]
            for val in R:
                p = p_choice(val)
                pha_img = pha(dnbr_img, plot=False, save=False)
                ppp_img = ppp(dnbr_img, plot=False, save=False)
                print(f'Per Pixel Analysis complete for basin {basin}, with a design storm of {int(val)*4} mm/hr.')
                #fig = fig_1x3(rgb_img, dnbr_img, ppp_img, savefig=True)
                #hist = fig_1x2(dnbr_img, ppp_img, savefig=False)
                #fig2 = fig_1x4(rgb_img, rgb_img2, dnbr_img, ppp_img, savefig=True)
                #fig1 = fig_6(rgb_img, rgb_img2, dnbr_img, ppp_img, p, savefig=True)
                fig = fig_8(rgb_img, rgb_img2, avg_dnbr_img, dnbr_img,
                            pha_img, ppp_img, p, savefig=False)
            print(f'Basin {basin} Debris Flow Likelihood analysis completed with all R values')
        #else:
            #print(f'Your input of {choice} is invalid. Quitting...')
           # break
                  

Input your choice.
                  1: Run all basins with R value = 24 mm/hr.
                  2: Run all basins with all R values.
                  1


DriverError: /mnt/nfs/lss/meerdink/home/skzebarth/masters/shapefiles/ma_data_bool.shp: No such file or directory

In [151]:
big_shape = '/mnt/nfs/lss/meerdink/home/skzebarth/masters/basin_data/basin_shp/'
#print(os.listdir(big_shape))
for filename in sorted(os.listdir(big_shape)):
    if filename.endswith('ma_data_bool.shp'):
        #b_name = filename.split('.')
        basin = 'all_basins'
        
        shape = gpd.read_file(os.path.join(big_shape, filename))
        rgb_img = rgb(dirImgLoc, shape, plot=False, save=True)
        rgb_img2 = rgb(dirImgLocPost, shape, plot=False, save=True)
        pre_fire_img = nbr_clip(dirImgLoc, shape, plot=False, save=True)
        print(f'Prefire imagery and basins NBR analysis complete for basin .')
        post_fire_img = nbr_clip(dirImgLocPost, shape, plot=False, save=True)
        print(f'Postfire imagery and basins NBR analysis complete for basin .')
        #avg_dnbr_img = avg_dnbr(pre_fire_img, post_fire_img, plot=False, save=True)
        dnbr_img = dnbr(pre_fire_img, post_fire_img, plot=False, save=True)
        print(f'DNBR analysis complete for basin .')

Basin ma_data_bool RGB combine complete
Basin ma_data_bool RGB combine complete
Prefire imagery and basins NBR analysis complete for basin .
Postfire imagery and basins NBR analysis complete for basin .
DNBR analysis complete for basin .
